In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = 'pyspark-shell'

In [2]:
#part2 la parte que vamos a convertir (la que guardamos). part1 la parte con la que vamos a hacer el join (el 'apoyo')
with open('/Kriptosare.class/tmp/part1.txt','r') as file:
    part1=int(file.read())        
with open('/Kriptosare.class/tmp/part2.txt','r') as file:
    part2=int(file.read())  
with open('/Kriptosare.class/tmp/turno_bloque.txt','r') as file:
    algo=file.read()
if part1>15:
    appname="part"+list(str(part1))[0]+'_'+list(str(part1))[1]+"+"+"part"+str(part2)+"+bloque_codigo"+algo
else:
    appname="part"+str(part1)+"+"+"part"+str(part2)+"+bloque_codigo"+algo

In [3]:
import findspark
findspark.init('/opt/spark')
from pyspark import SparkContext,SparkConf

#with cluster
#conf = (SparkConf()
#         .setMaster("spark://10.200.5.39:7077")
#         .set("spark.driver.host","10.200.5.39") 
#         .set("spark.executor.memory","58g")
#         .set('spark.driver.memory', '58G')
#         .setAppName(appname))
#sc = SparkContext(conf=conf)

In [4]:
path = "Kriptosare.class/analysis"
pathDir ="transaction_data_convert_complete/"

In [ ]:
import pyspark
import binascii
from pyspark.sql import SQLContext
from functools import reduce
#import pygraphviz
import pyspark.sql.functions as f
from IPython.display import Image
#from networkx.drawing.nx_pydot import write_dot
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyspark.sql import Row
import math
import subprocess

sqlContext = SQLContext(sc)

In [ ]:
"""from IPython.display import HTML, Javascript, display

def initialize():
    display(HTML(
        '''
            <script>
                code_show = false;
                function restart_run_all(){
                    IPython.notebook.kernel.restart();
                    setTimeout(function(){
                        IPython.notebook.execute_all_cells();
                    }, 15000)
                }
            document.getElementById("demo").innerHTML = restart_run_all();
            </script>
            <p id="demo"></p>  

        '''
    ))"""

In [ ]:
from IPython.display import display_html
def initialize():
    display_html("<script>IPython.notebook.kernel.restart()</script>",raw=True)
    sc.stop()
    !sleep 15
    !jupyter nbconvert --execute transaction_data_convert_auto.ipynb --ExecutePreprocessor.timeout=-1

In [ ]:
class load_parts():   
    def __init__(self,part1,part2):
        if part1<15:
            name1=str(part1)
        else:
            name1=list(str(part1))[0]+'_'+list(str(part1))[1]

            
        self.dfs_list=[]
        self.dfs_list.append(sqlContext.read\
                     .format("csv")\
                     .option("header", "true")\
                     .option("inferSchema", "true")\
                     .load(path+"bitcoin/transaction_part"+name1))
                             
        self.dfs_list.append(sqlContext.read\
                     .format("csv")\
                     .option("header", "true")\
                     .option("inferSchema", "true")\
                     .load(path+"bitcoin/transaction_part"+str(part2)))
    
    def load(self):        
        return self.dfs_list


In [ ]:
dfs_list=load_parts(part1,part2).load()

In [ ]:
#Por bloques. 

def generic_initial_df_convert(dfs_list):
    #hasta aquí
    with open('/Kriptosare.class/tmp/turno_bloque.txt','r') as file:
        turno_bloque=file.read()
    if turno_bloque=='1':
        bloque1(dfs_list)
        initialize()
        return
    elif turno_bloque=='2':
        bloque2(dfs_list)                
        initialize()
        return
    else:
        bloque3()
        if part1==11 and part2==12:
            with open('/Kriptosare.class/tmp/checkpoint2.txt','w') as file:
                file.write('finished')
            os._exit(00)
            return 
        initialize()
        return





def initial_df_convert(df_transactions_part):
    df_out=df_transactions_part.select('vin_txid','vin_vout','tx_hash','address','timestamp','amount')
    df_out=df_out.withColumnRenamed('address','address_llegada')
    df_qin=df_transactions_part.select('address','vout_idx','tx_hash').withColumnRenamed('tx_hash','tx_hash1')
    df_qin=df_qin.withColumnRenamed('address','address_salida').dropDuplicates() 
    return df_qin\
        .join(df_out, (df_out.vin_txid == df_qin.tx_hash1)&(df_out.vin_vout==df_qin.vout_idx),how='right')\
        .drop('vin_txid','vin_vout','vout_idx','tx_hash1').sort('timestamp').dropDuplicates() 


def initial_df_convert_no_null(df_transactions_part):
    df_out=df_transactions_part.select('vin_txid','vin_vout','tx_hash','address','timestamp','amount')
    df_out=df_out.withColumnRenamed('address','address_llegada')
    df_qin=df_transactions_part.select('address','vout_idx','tx_hash').withColumnRenamed('tx_hash','tx_hash1')
    df_qin=df_qin.withColumnRenamed('address','address_salida').dropDuplicates()     
    return df_qin\
        .join(df_out, (df_out.vin_txid == df_qin.tx_hash1)&(df_out.vin_vout==df_qin.vout_idx))\
        .drop('tx_hash1','vout_idx').sort('timestamp').dropDuplicates()  


def bloque1(dfs_list):
    with open('/Kriptosare.class/tmp/bloque_actual.txt','w') as file:
                file.write('bloque1') 
    df_prueba=initial_df_convert(dfs_list[1].filter(f.col('coinbase')==True))\
            .union(initial_df_convert_no_null(dfs_list[1]).drop('vin_txid','vin_vout'))
    
    df_miner=dfs_list[1].filter(f.col('coinbase')==True).withColumnRenamed('address','address1')\
    .withColumnRenamed('tx_hash','tx_hash1')
                #print(df_miner.count(),1)
            
        
    df_null_as=initial_df_convert(dfs_list[1]).filter(f.col('address_salida').isNull())
                #print(df_null_as.count())
            
    
    df_null_as=df_null_as.join(df_miner,(df_miner.tx_hash1==df_null_as.tx_hash)\
                &(df_miner.address1==df_null_as.address_llegada),how='left')
        
        
    df_null_as=df_null_as.filter(df_null_as.address1.isNull())\
                    .dropDuplicates().select('address_llegada','tx_hash').withColumnRenamed('tx_hash','tx_hash1')
        
                #df_null_aux=df_null_as


    df_null_as=df_null_as.join(dfs_list[1],(df_null_as.address_llegada==dfs_list[1].address)\
                                    &(df_null_as.tx_hash1==dfs_list[1].tx_hash)).drop('tx_hash1','address_llegada')\
                .dropDuplicates()
        
                #print(df_null_as.count(),2)
    with open('/Kriptosare.class/tmp/checkpoint1.txt','w') as file:
                file.write('primer parquet sin hacer') 
    try:
        df_prueba.write.parquet(path+pathDir+"transaction_data_convert_part"+str(part2)+".parquet")
    except Exception:
        df_save_copy=sqlContext.read.parquet(path+pathDir+"transaction_data_convert_part"+str(part2)+".parquet")
        df_save_copy.write.parquet(path+pathDir+"transaction_data_convert_part_backup.parquet",mode='overwrite')
    
    with open('/Kriptosare.class/tmp/checkpoint1.txt','w') as file:
                file.write('empieza el df_null_as') 
    
    df_null_as.write.parquet(path+"transaction_data_convert_complete_auxiliars/df_null_as.parquet",mode='overwrite')
    
    with open('/Kriptosare.class/tmp/checkpoint1.txt','w') as file:
                file.write('df_null_as escrito')
    
    with open('/Kriptosare.class/tmp/turno_bloque.txt','w') as file:
        file.write('2')

    
def bloque2(dfs_list):
    
    df_save_copy=sqlContext.read.parquet(path+pathDir+"transaction_data_convert_part"+str(part2)+".parquet")
    df_save_copy.write.parquet(path+pathDir+"transaction_data_convert_part_backup.parquet",mode='overwrite')
    
    df_null_as=sqlContext.read.parquet(path+"transaction_data_convert_complete_auxiliars/df_null_as.parquet")
    
    with open('/Kriptosare.class/tmp/bloque_actual.txt','w') as file:
                file.write('bloque2') 
    
            

    df_aux=initial_df_convert_no_null(dfs_list[0].union(df_null_as))
    df_aux.write.parquet(path+"transaction_data_convert_complete_auxiliars/df_aux.parquet",mode='overwrite')
    with open('/Kriptosare.class/tmp/turno_bloque.txt','w') as file:
            file.write('3')

def bloque3():
    with open('/Kriptosare.class/tmp/bloque_actual.txt','w') as file:
                file.write('bloque3') 
    dfs_renamed=dfs_list[1].withColumnRenamed('address','address1').withColumnRenamed('tx_hash','tx_hash1')\
                .withColumnRenamed('timestamp','timestamp1').withColumnRenamed('amount','amount1')     
    
    df_aux=sqlContext.read.parquet(path+"transaction_data_convert_complete_auxiliars/df_aux.parquet")
        
    df_aux=df_aux.join(dfs_renamed,(dfs_renamed.tx_hash1==df_aux.tx_hash)\
                &(dfs_renamed.address1==df_aux.address_llegada)&(dfs_renamed.vin_vout==df_aux.vin_vout)\
                &(dfs_renamed.vin_txid==df_aux.vin_txid)&(dfs_renamed.amount1==df_aux.amount))\
                .select('address_salida','tx_hash','address_llegada','timestamp','amount')
                    #print(df_aux.count(),3)
            
    with open('/Kriptosare.class/tmp/checkpoint2.txt','w') as file:
                file.write('segundo parquet sin hacer')                    
    df_aux.write.parquet(path+pathDir+"transaction_data_convert_part"+str(part2)+".parquet",mode='append')
    
    
    
    with open('/Kriptosare.class/tmp/checkpoint2.txt','w') as file:
                file.write('segundo parquet hecho')
    
    if part1==part2-1:
        with open('/Kriptosare.class/tmp/part1.txt','w') as file:
            file.write('1')
                
        with open('/Kriptosare.class/tmp/part2.txt','w') as file:
            file.write(str(part2+1))
                
        with open('/Kriptosare.class/tmp/turno_bloque.txt','w') as file:
            file.write('1')
    else:
        with open('/Kriptosare.class/tmp/part1.txt','w') as file:
            file.write(str(part1+1))
        with open('/Kriptosare.class/tmp/turno_bloque.txt','w') as file:
            file.write('2')

    

In [ ]:
generic_initial_df_convert(dfs_list)